In [0]:
import os
import csv
import re
import logging
import traceback
import optparse
import dedupe
from unidecode import unidecode
import pandas as pd
import json
import os
import sys
import itertools

logger = logging.getLogger("Dedupe")
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
#logger.setLevel(logging.INFO)
logger.setLevel(logging.DEBUG)
logging.getLogger("py4j").setLevel(logging.ERROR)

def preProcess(column):
    column = unidecode(column)
    column = re.sub('  +', ' ', column)
    column = re.sub('\n', ' ', column)
    column = column.strip().strip('"').strip("'").lower().strip()
    if not column:
        column = None
    return column

def readData(filename):
    data_d = {}
    logger.debug("Reading the input dataset with CSV reader")
    with open(filename) as f:
        reader = csv.DictReader(f)
        for pos, row in enumerate(reader):
            clean_row = [(k, preProcess(v)) for (k, v) in row.items()]
            clean_row.append(('unq_id', '{value}'.format(value=pos)))
            data_d[pos] = dict(clean_row)
    return data_d

def getFields(dedupe_fields=None, missingCol = None):
    logger.debug("formatting the fields to be used in dedupe process")
    listof = list()
    if ((dedupe_fields !=None) and (missingCol != None)):
        for k,v in dedupe_fields.items():
            dictof = dict()
            dictof['field'] = k
            dictof['type'] = v
            listof.append(dictof)
        for k,v in missingCol.items():
            dictof = dict()
            dictof['field'] = k
            dictof['type'] = v
            dictof['has missing'] = True
            listof.append(dictof)
        return listof
    else:
        if dedupe_fields == None:
            for k,v in missingCol.items():
                dictof = dict()
                dictof['field'] = k
                dictof['type'] = v
                dictof['has missing'] = True
                listof.append(dictof)
            return listof
        else:
            for k,v in dedupe_fields.items():
                dictof = dict()
                dictof['field'] = k
                dictof['type'] = v
                listof.append(dictof)
            return listof
            
def dedupeFormatMap(datafromUI):
    logger.debug("Formatting the data received from UI")
    dictUIData=json.loads(datafromUI)
    labeled_se = {'match' : [],'distinct' : []}
    for key,value in dictUIData.items():
        for k,v in value.items():
            if k=='yes':
                labeled_se['match'].append(tuple(v))
                logger.debug("Filtering the pair to mathed response list which is yes")
        else:
            labeled_se['distinct'].append(tuple(v))
            logger.debug("Filtering the pair to un-mathed response list which is no")
    return labeled_se
  
def dedupeTrainingFields(inputdf):
    input_panda_df=inputdf
    fields_dict=input_panda_df.dtypes.to_dict()
    tot_row_count=len(input_panda_df)
    logger.debug("Total rows available in the input dataset: "+str(tot_row_count))
    total_columns=len(input_panda_df.columns)
    logger.debug("Total columns available in the input dataset: "+str(total_columns))
    num_fields=round(total_columns*0.6)
    logger.debug("Total fields to be captured for dedupe process: "+str(num_fields))
    unique_count_df=input_panda_df.T.apply(lambda x: x.nunique(), axis=1).to_frame()
    field_count_df=unique_count_df.reset_index().rename(columns={'index':'columns',0:'count'}).sort_values('count',ascending= False)
    max_count_field=field_count_df.max().tolist()
    field_df=field_count_df.head(num_fields)
    dedupe_configFields=field_df['columns'].tolist()
    dedupe_configFields.remove("unq_id")
    fields_dict_1 = {k:v for (k,v) in fields_dict.items() if k in dedupe_configFields}
    dedupeF_withTypes={k:('Exact' if v=='int64' else 'Exact' if v=='float64' else 'String') for (k,v) in fields_dict_1.items()}
    isMissingFields_df=input_panda_df[dedupe_configFields].T.apply(lambda x: x.isnull().sum(), axis=1).to_frame().reset_index().rename(columns={'index':'columns',0:'nullCount'})
    isMissingFields_df= isMissingFields_df[isMissingFields_df['nullCount'] > 0]
    isMissingFields= isMissingFields_df['columns'].tolist()
    isMissingFields_dict= {k:v for (k,v) in fields_dict.items() if k in isMissingFields}
    isMissingFields_withTypes= {k:('Exact' if v=='int64' else 'Exact' if v=='float64' else 'String') for (k,v) in isMissingFields_dict.items()}
    dedupeF_withTypes_notmissing={k:v for (k,v) in dedupeF_withTypes.items() if k not in isMissingFields}
    dedupe_fields=getFields(dedupe_fields=dedupeF_withTypes_notmissing, missingCol = isMissingFields_withTypes)
    return dedupe_fields
  
def activeLearingSetup(inputPath):
    logger.debug("Used file for dedupe process "+ str(inputPath))
    try:
        firstpos=inputPath.rfind("/")
        lastpos=len(inputPath)
        input_file=inputPath
        foup=inputPath[firstpos+1:lastpos].split(".")[0]
        logger.debug("Exact name without ext of file: "+ str(foup))
        processedFilepath='/webmnt/processedFiles/'
        trainingFilepath='/webmnt/trainingFiles/'
        output_file_dup = processedFilepath+foup+'_with_dup_output.csv'
        logger.debug("processed file with duplicates stored at: "+ str(output_file_dup))
        output_file_unq = processedFilepath+foup+'_with_unq_output.csv'
        logger.debug("processed file with unq data stored at: "+ str(output_file_unq))
        settings_file = trainingFilepath+foup+'_learned_settings'
        logger.debug("setting file at: "+ str(settings_file))
        training_file = trainingFilepath+foup+'_training.json'
        logger.debug("training file at: "+ str(training_file))
        #output_file_dup=foup+'_with_dup_output.csv' #For Local Run
        #output_file_unq=foup+'_with_unq_output.csv' #For Local Run
        #settings_file=foup+'_learned_settings' #For Local Run
        #training_file=foup+'_training.json' #For Local Run
        read_panda_df=pd.read_csv(input_file)
        logger.info("Reading data from input dataset")
        read_panda_df['unq_id']=read_panda_df.index
        input_panda_df=read_panda_df
        dedupe_fields=dedupeTrainingFields(input_panda_df)
        logger.info("Getting the training field details ")
        data_d = readData(input_file)
        fields = dedupe_fields
        deduper = dedupe.Dedupe(fields)
        logger.debug("specifying training field for dedupe process"+str(fields))
        deduper.prepare_training(data_d)
        logger.info("Preparing training data for dedupe process")
        fieldList=[(lambda x: x['field'])(x) for x in fields]   
        pairRows={}  
        for turns in range(5):
            pairs=deduper.uncertain_pairs()
            for ele in pairs:
                pairRows["pair"+str(turns)]=list(ele)
        pairRows['display']=fieldList
        logger.debug("Appending display fields with the pairs")
        jsonToUI=json.dumps(pairRows)
        if os.path.exists(settings_file):
            logger.debug("setting file available performimg static dedupe")
            return processStaticDedupe(input_file,output_file_dup,output_file_unq,settings_file,training_file,input_panda_df,fields,data_d)
        else:
            logger.debug("returning pairs to UI for selection to generate training data")
            return jsonToUI
    except:
        logger.error(str(traceback.print_exc()))
        raise ValueError("ERROR in "+ __name__+ " details are "+ str(traceback.format_exc()))

        
def processStaticDedupe(input_file,output_file_dup,output_file_unq,settings_file,training_file,input_panda_df,fields,data_d):
    logger.debug("Static dedupe process started")
    logger.debug("reading from"+str(settings_file))
    with open(settings_file, 'rb') as f:
         deduper = dedupe.StaticDedupe(f)
    logger.info("Clustering...........................")
    clustered_dupes = deduper.partition(data_d, 0.5)
    duplicate_sets=len(clustered_dupes)
    logger.debug("duplicate_sets"+str(duplicate_sets))
    cluster_membership = {}
    for cluster_id, (records, scores) in enumerate(clustered_dupes):
        for record_id, score in zip(records, scores):
            cluster_membership[record_id] = {
                "Cluster ID": cluster_id,
                "confidence_score": score
            }
    
    cldf=pd.DataFrame.from_dict(cluster_membership, orient='index').reset_index().rename(columns={'index':'unq_id'})
    alldf=input_panda_df.merge(cldf, on ='unq_id',how='left')
    alldf['count']=alldf.groupby(['Cluster ID'])['Cluster ID'].transform('count')
    UniqueDf=alldf.drop_duplicates(subset ="Cluster ID", keep = 'first', inplace = False).drop('count', axis = 1)
    logger.debug("Count of rows after dropping duplicates"+str(len(UniqueDf)))
    UniqueDf.to_csv(output_file_unq, index=False)
    logger.debug("Writing unique dataset after dropping duplicates"+str(output_file_unq))
    mask = alldf['count'] > 1
    All_MultiDf = alldf[mask].drop('count', axis = 1)
    logger.debug("Count of only duplicate datasets"+str(len(All_MultiDf)))
    All_MultiDf.to_csv(output_file_dup, index=False)
    logger.debug("Writing only duplicate datasets to "+str(output_file_dup))
    #clustercountdf=alldf[['Cluster ID','count']].drop_duplicates().sort_values('count',ascending= False)
    return json.dumps({"duplicate_sets":duplicate_sets})
    
def processLabled(inputPath,jsonfromUI):
    logger.debug("Process started with file "+ str(inputPath))
    try:
        firstpos=inputPath.rfind("/")
        lastpos=len(inputPath)
        input_file=inputPath
        foup=inputPath[firstpos+1:lastpos].split(".")[0]
        logger.debug("Exact name without ext of file: "+ str(foup))
        processedFilepath='/webmnt/processedFiles/'
        trainingFilepath='/webmnt/trainingFiles/'
        output_file_dup = processedFilepath+foup+'_with_dup_output.csv'
        logger.debug("processed file with duplicates stored at: "+ str(output_file_dup))
        output_file_unq = processedFilepath+foup+'_with_unq_output.csv'
        logger.debug("processed file with unq data stored at: "+ str(output_file_unq))
        settings_file = trainingFilepath+foup+'_learned_settings'
        logger.debug("setting file at: "+ str(settings_file))
        training_file = trainingFilepath+foup+'_training.json'
        logger.debug("training file at: "+ str(training_file))
        #output_file_dup=foup+'_with_dup_output.csv' #For Local Run
        #output_file_unq=foup+'_with_unq_output.csv' #For Local Run
        #settings_file=foup+'_learned_settings' #For Local Run
        #training_file=foup+'_training.json' #For Local Run
        read_panda_df=pd.read_csv(input_file)
        logger.info("Reading data from input dataset")
        read_panda_df['unq_id']=read_panda_df.index
        input_panda_df=read_panda_df
        dedupe_fields=dedupeTrainingFields(input_panda_df)
        logger.info("Getting the training field details ")
        data_d = readData(input_file)
        fields = dedupe_fields
        deduper = dedupe.Dedupe(fields)
        logger.debug("specifying training field for dedupe process"+str(fields))
        deduper.prepare_training(data_d)
        logger.info("Preparing training data for dedupe process")
        logger.debug("starting active labeling...")
        labeled_selection=dedupeFormatMap(jsonfromUI)
        deduper.mark_pairs(labeled_selection)
        logger.debug("Training the model")
        deduper.train()
        logger.debug("Writing training file to :"+str(training_file))
        with open(training_file, 'w') as tf:
            deduper.write_training(tf)
            
        logger.debug("Writing settings file to :"+str(settings_file))
        with open(settings_file, 'wb') as sf:
            deduper.write_settings(sf)
        
        logger.info("Clustering...........................")
        clustered_dupes = deduper.partition(data_d, 0.5)
        duplicate_sets=len(clustered_dupes)
        logger.debug("duplicate_sets"+str(duplicate_sets))
        cluster_membership = {}
        for cluster_id, (records, scores) in enumerate(clustered_dupes):
            for record_id, score in zip(records, scores):
                cluster_membership[record_id] = {
                    "Cluster ID": cluster_id,
                    "confidence_score": score
                }
        
        cldf=pd.DataFrame.from_dict(cluster_membership, orient='index').reset_index().rename(columns={'index':'unq_id'})
        alldf=input_panda_df.merge(cldf, on ='unq_id',how='left')
        alldf['count']=alldf.groupby(['Cluster ID'])['Cluster ID'].transform('count')
        UniqueDf=alldf.drop_duplicates(subset ="Cluster ID", keep = 'first', inplace = False).drop('count', axis = 1)
        logger.debug("Count of rows after dropping duplicates"+str(len(UniqueDf)))
        UniqueDf.to_csv(output_file_unq, index=False)
        logger.debug("Writing unique dataset after dropping duplicates"+str(output_file_unq))
        mask = alldf['count'] > 1
        All_MultiDf = alldf[mask].drop('count', axis = 1)
        logger.debug("Count of only duplicate datasets"+str(len(All_MultiDf)))
        All_MultiDf.to_csv(output_file_dup, index=False)
        logger.debug("Writing only duplicate datasets to "+str(output_file_dup))
        #clustercountdf=alldf[['Cluster ID','count']].drop_duplicates().sort_values('count',ascending= False)
        return json.dumps({"duplicate_sets":duplicate_sets})
    except:
        logger.error(str(traceback.print_exc()))
        raise ValueError("ERROR in "+ __name__+ " details are "+ str(traceback.format_exc()))

def FieldsUsedforSetup(inputPath):
    logger.setLevel(level=logging.DEBUG)
    fieldPublish={}
    input_file=inputPath
    try:
      logger.debug("Reading data from file : "+str(inputPath))
      read_panda_df=pd.read_csv(input_file)
      logger.info("Adding unq_id column to the dataset")
      read_panda_df['unq_id']=read_panda_df.index
      input_panda_df=read_panda_df
      logger.info("Filtering the fields to be used for dedupe process")
      dedupe_fields=dedupeTrainingFields(input_panda_df)
      fields = dedupe_fields
      fieldList=[(lambda x: x['field'])(x) for x in fields]
      logger.info("Dedupe process filtered fields: "+str(fieldList))
      fieldPublish['display']=fieldList
      jsonToUI=json.dumps(fieldPublish)
      return jsonToUI
    except:
      logger.error(str(traceback.print_exc()))
      raise ValueError("ERROR in "+ __name__+ " while generating the column list for deduplication from the input dataset "+ str(traceback.format_exc()))
    finally:
      logger.info("End of fields retrival for dedupilcation process")